# **0. 해커톤 진행 주의사항**

**1)  개발 관련 주의사항**
*   [1. 초기 환경 설정]은 절대 수정하지 말 것
*   모든 구현은 [2. 데이터 전처리] 및 [3.모델 생성]에서만 진행
*   [4. 모델 저장]에서 team_name 변수 변경 (예.`team_name = 'team01'`)
 *    트레이닝 중간에 checkpoint를 활용하여 모델을 저장한 경우에도 파일 이름 양식 통일 필수
*   Colab 사용중 실수로 데이터 손실이 발생할 수도 있으니 중간 결과값을 github에 업로드 
 *    "런타임->모든 런타임 재설정"은 절대 누르지 말 것 (저장한 모델 데이터가 모두 삭제됨)
*   효율적인 구현 및 테스팅을 위해 GPU 가속 기능 활성화
 *    "런타임 -> 런타임 유형변경 -> 하드웨어 가속기 -> GPU 설정"
*   주석을 최대한 자세히 작성
*   Keras API 관련하여 [Keras Documentation](https://keras.io/) 참조

**2) 제출 관련 주의사항**
*  제출물
 *  소스코드 (hackathon_teamXX.ipynb)
 *  컴파일된 모델 파일 (model_entire_teamXX.h5)
 *  모델 발표 자료 
* 제출 기한: **오후 6시 (단, 발표자료는 12시)**
* 제출 방법: [GitHub README](https://github.com/cauosshackathonta/2021_cau_oss_hackathon/) 참조

 
**3) 평가 관련 주의사항**
*  모델 성능 = 두개의 테스트 데이터 셋 분류 정확도에 대한 weighted sum
 *  model.evaluate(x_test1, y_test1) + model.evaluate(x_test2, y_test2) * 2
*  제출된 모델들의 테스트 데이터 셋 분류 정확도를 기준으로 수상작 결정
*  수상 후보들에 대해서는 소스코드를 기반으로 모델 재검증 
 
**4) 수상 실격 사유**
*  유사한 소스코드가 적발될 경우
*  Pre-trained 모델을 사용한 경우 (transfer learning 포함)
*  소스코드와 제출된 모델이 상이한 경우
*  개발 관련 주의사항을 지키지 않은 경우
 *  예: [초기 환경 설정]을 수정한 경우
*  데이터 셋을 변조한 경우
 *  예. 테스트 데이터 셋을 트레이닝 데이터 셋에 포함하여 모델 생성 
*  주석이 소스코드와 맞지 않거나 미비할 경우






# **1. 초기 환경 설정**



In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras 및 관련 라이브러리 임포트
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from tensorflow import keras
from keras.utils import np_utils

# 데이터셋 로드 (Training dataset: MNIST, test dataset: MNIST_corrupted)
train_ds = tfds.load('mnist', split='train', shuffle_files=False, batch_size=-1)
test_ds1 = tfds.load('mnist_corrupted/scale', split='test', shuffle_files=False, batch_size=-1)
test_ds2 = tfds.load('mnist_corrupted/fog', split='test', shuffle_files=False, batch_size=-1)
test_ds3 = tfds.load('mnist_corrupted/zigzag', split='test', shuffle_files=False, batch_size=-1)

train_ds = tfds.as_numpy(train_ds)
test_ds1 = tfds.as_numpy(test_ds1)
test_ds2 = tfds.as_numpy(test_ds2)
test_ds3 = tfds.as_numpy(test_ds3)

x_train, y_train = train_ds['image'], train_ds['label']
x_test = np.append(np.append(test_ds1['image'], test_ds2['image'], 0), test_ds3['image'], 0);
y_test = np.append(np.append(test_ds1['label'], test_ds2['label'], 0), test_ds3['label'], 0);

# #분류를 위해 클래스 벡터를 바이너리 매트릭스로 변환
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

# # 총 클래스 개수
num_classes = y_train.shape[1]
input_shape = x_train.shape[1:]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to ~/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

# **2. 데이터 전처리**



In [ ]:
# 데이터 전처리 (예: normalization)
# 원본 데이터와 전처리 후 데이터를 구분하기 위해, 변수명 x_train_after, x_test1_after, x_test2_after를 변경하지 말 것
x_train_after = x_train / 255.0
x_test_after = x_test / 255.0

# **3. 모델 생성**

In [ ]:
# 순차 모델 생성 (가장 기본구조)
model = keras.Sequential()

# Flatten layer: 32 x 32 x 3 image를 3072개의 1D vector input으로 변환
model.add(keras.layers.Flatten(input_shape=input_shape))

# 1st hidden layer: fully-connected layer
# (# of inputs = 3072, # of outputs = 512, actication fuction = relu)
model.add(keras.layers.Dense(512, activation=tf.nn.relu))

# 2nd hidden layer: fully-connected layer 
# (# of inputs = 512, # of outputs = 256, actication fuction = relu)
model.add(keras.layers.Dense(256, activation=tf.nn.relu))

# 3rd hidden layer: fully-connected layer 
# (# of inputs = 256, # of outputs = 64, actication fuction = relu)
model.add(keras.layers.Dense(64, activation=tf.nn.relu))

# Output layer: fully-connected layer 
# (# of inputs = 64, # of outputs = 10, actication fuction = softmax)
model.add(keras.layers.Dense(num_classes, activation=tf.nn.softmax))

# 모델 컴파일
# optimizer: 모델을 업데이트 하는 방식
# loss: 모델의 정확도를 판단하는 방식
# metrics: 트레이닝 및 테스팅 성능 모니터링을 위한 평가지표
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


# 체크포인트 생성
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath='/content/checkpoint_entire_best.h5', monitor='val_accuracy', verbose=1, save_weight_only=False, save_best_only=True, mode='auto')

# 모델 트레이닝
# batch_size: 전체 데이터셋 중 몇개씩 학습시킬 것인지
# epoch: 학습에 전체 데이터셋이 총 몇번 이용될 것인지
# shuffle: 학습전에 트레이닝 데이터셋을 랜덤하게 섞을 것인지
# validation_data: 중간 성능 검증에 사용할 data set (x_test1_after, x_test2_after, 혹은 둘을 merge해서 사용)
model.fit(x_train_after, y_train, batch_size = 128, epochs = 10, shuffle=True, callbacks=[cp_callback], validation_data=(x_test_after, y_test))

Epoch 1/10
463/469 [============================>.] - ETA: 0s - loss: 0.2307 - accuracy: 0.9319
Epoch 1: val_accuracy improved from -inf to 0.46680, saving model to /content/checkpoint_entire_best.h5
469/469 [==============================] - 3s 5ms/step - loss: 0.2292 - accuracy: 0.9323 - val_loss: 2.6290 - val_accuracy: 0.4668
Epoch 2/10
462/469 [============================>.] - ETA: 0s - loss: 0.0840 - accuracy: 0.9737
Epoch 2: val_accuracy improved from 0.46680 to 0.49217, saving model to /content/checkpoint_entire_best.h5
469/469 [==============================] - 2s 4ms/step - loss: 0.0837 - accuracy: 0.9738 - val_loss: 2.0819 - val_accuracy: 0.4922
Epoch 3/10
467/469 [============================>.] - ETA: 0s - loss: 0.0537 - accuracy: 0.9834
Epoch 3: val_accuracy improved from 0.49217 to 0.51677, saving model to /content/checkpoint_entire_best.h5
469/469 [==============================] - 2s 5ms/step - loss: 0.0535 - accuracy: 0.9834 - val_loss: 2.5015 - val_accuracy: 0.5168
E

# **4. 모델 저장**

In [ ]:
save_path = '/content/'
team_name = 'teamXX'

# 트레이닝된 전체 모델을 저장합니다.
model.save(save_path +  'model_entire_'+ team_name + '.h5')

# **5. 모델 로드 및 평가**

In [ ]:
save_path = '/content/'
team_name = 'teamXX'

model = keras.models.load_model(save_path + 'model_entire_' + team_name + '.h5')

model.evaluate(x_test_after, y_test)